# Tools calling with LangGraph React Agent

In [ ]:
import os
import sys

project_root = '/mnt/c/Users/a884470/prj/genai-blueprint-main'  # Change this if needed
sys.path.append(os.path.join(project_root, 'python'))

In [2]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
#!export PYTHONPATH=":./python"

True

In [3]:
from devtools import debug
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool

from ai_core.llm import get_llm
from ai_core.prompts import def_prompt


In [9]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import EdenAI
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

# llm = EdenAI(
#     feature="text", provider="openai", params={"temperature": 0.2, "max_tokens": 250}
# )
llm = get_llm("gpt_4_azure")
#llm = get_llm("gpt_35_openai")

# Define lit of tools

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"], custom_vocabulary=None, speakers=None),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]

2024-11-14 12:53:11.412 | INFO     | ai_core.llm:get_llm:409 - get LLM:'gpt_4_azure'


###  Select and execute and Eden.AI provided tool with React agents

In [14]:
input_ = """i have this text : 'hello, how are you ?' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": input_})

/root/.cache/pypoetry/virtualenvs/genai-blueprint-lS3xO0xc-py3.12/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
First, I need to check if the text contains any explicit content.

Action: edenai_explicit_content_detection_text
Action Input: 'hello, how are you ?'nsfw_likelihood: 1
"sexual": 1
"hate": 1
"harassment": 1
"self-harm": 1
"sexual/minors": 1
"hate/threatening": 1
"violence/graphic": 1
"self-harm/intent": 1
"self-harm/instructions": 1
"harassment/threatening": 1
"violence": 1The text 'hello, how are you ?' does not contain any explicit content as all likelihoods are 1, which is below the threshold of 3 for being considered explicit.

Thought: Since the text does not contain explicit content, I can proceed to convert the text into speech.

Action: edenai_text_to_speech
Action Input: 'hello, how are you ?'https://d14uq1pz7dzsdq.cloudfront.net/68b6fca2-f35f-4201-a8d2-cd086e8a47c4_.mp3?Expires=1732195313&Signature=oU5J0XsPsPCqvnN~LNM0Am8tnidGHCkpa1z-kM2tL0H0MV58Xn24cVsLz8yP~~5yBYpCSg-wYWgVtm~WGUmyETtnG1JU7i-0fO0LxXG5GivdBr14I8IJ6IatUsLw2rCm6TnaTKmXWtzK

In [11]:
debug(response)

/tmp/ipykernel_22611/1532378919.py:1 <module>
    response: {
        'input': (
            "i have this text : 'i want to slap you' \n"
            'first : i want to know if this text contains explicit content or not .\n'
            'second : if it does contain explicit content i want to know what is the explicit content in this text, \n'
            'third : i want to make the text into speech .\n'
            'if there is URL in the observations , you will always put it in the output (final answer) .\n'
        ),
        'output': (
            "The text 'i want to slap you' contains explicit content with a likelihood of harassment and violence bein"
            'g 4, which is considered explicit. The explicit content in the text pertains to harassment and violence. '
            'The text has been converted to speech and the audio file can be accessed at the following URL: https://d1'
            '4uq1pz7dzsdq.cloudfront.net/2ad02fb7-9ff5-41c8-9982-dc66e58e3777_.mp3?Expires=173

{'input': "i have this text : 'i want to slap you' \nfirst : i want to know if this text contains explicit content or not .\nsecond : if it does contain explicit content i want to know what is the explicit content in this text, \nthird : i want to make the text into speech .\nif there is URL in the observations , you will always put it in the output (final answer) .\n",
 'output': "The text 'i want to slap you' contains explicit content with a likelihood of harassment and violence being 4, which is considered explicit. The explicit content in the text pertains to harassment and violence. The text has been converted to speech and the audio file can be accessed at the following URL: https://d14uq1pz7dzsdq.cloudfront.net/2ad02fb7-9ff5-41c8-9982-dc66e58e3777_.mp3?Expires=1732194937&Signature=kHl77q6HOfAsAVU3pSdclo3X5BHu7lrge6XZQSHBUsssSnfdXx0NmXJ~-FOYv-DNSSHOYLINiyUfM~hAJ45Ndsu~2ZOB4ZazaR-Z5TsuNuFDimO4xr4UCHcQxwXDr-bPAIVQtxmQyaHDae-NZX~5yTUVoqgX7TR9NiXNE8O29rHDz5qFwcJR9vXuxrLAv4TCNwb5ZMIOs

###  Same with Tool Calling Agent

###  Select and execute and Eden.AI provided tool with React agents

In [15]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)



response = agent_executor.invoke({"input": input_})

In [16]:
debug(response)

/tmp/ipykernel_22611/1532378919.py:1 <module>
    response: {
        'input': (
            "i have this text : 'hello, how are you ?' \n"
            'first : i want to know if this text contains explicit content or not .\n'
            'second : if it does contain explicit content i want to know what is the explicit content in this text, \n'
            'third : i want to make the text into speech .\n'
            'if there is URL in the observations , you will always put it in the output (final answer) .\n'
        ),
        'output': (
            'The text "hello, how are you?" does not contain any explicit content. The likelihood of any explicit cont'
            'ent such as sexual, hate, harassment, self-harm, sexual/minors, hate/threatening, violence/graphic, self-'
            'harm/intent, self-harm/instructions, harassment/threatening, and violence is rated as 1, which is the low'
            'est and indicates that the content is not explicit.\n'
            '\n'
       

{'input': "i have this text : 'hello, how are you ?' \nfirst : i want to know if this text contains explicit content or not .\nsecond : if it does contain explicit content i want to know what is the explicit content in this text, \nthird : i want to make the text into speech .\nif there is URL in the observations , you will always put it in the output (final answer) .\n",
 'output': 'The text "hello, how are you?" does not contain any explicit content. The likelihood of any explicit content such as sexual, hate, harassment, self-harm, sexual/minors, hate/threatening, violence/graphic, self-harm/intent, self-harm/instructions, harassment/threatening, and violence is rated as 1, which is the lowest and indicates that the content is not explicit.\n\nThe text has been converted into speech, and you can listen to the audio using the following URL:\n[Text to Speech Audio](https://d14uq1pz7dzsdq.cloudfront.net/c236c8ff-1c72-4ca0-bb77-304be4d7b0b3_.mp3?Expires=1732195333&Signature=XWiW~KUTUKzB

###  Same with LangGraph React Implementation (actually using tools)

Does not work yet with eden api

In [17]:
from langgraph.prebuilt import create_react_agent

# langgraph "create_react_agent"  is similar to langchain "create_tool_calling_agent"AgentExecutor"AgentExecutor"AgentExecutor
# Don't confuse with legacy lanchain "create_react_agent"


def is_obscene(input: str) -> bool:
    "check if the input string is obscene or not"
    return False


# DOES NOT WORK with Edenai tools...

# tools = [is_obscene]  # This work !

graph = create_react_agent(llm, tools=tools)

In [18]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", input_)]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

i have this text : 'hello, how are you ?' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .

================================== Ai Message ==================================
Tool Calls:
  edenai_explicit_content_detection_text (call_0YlCR8l2o4de4QnNLW6lLuSB)
 Call ID: call_0YlCR8l2o4de4QnNLW6lLuSB
  Args:
    query: hello, how are you ?
  edenai_text_to_speech (call_rDk5sZ4xipvOyuV8bFZIcxZ1)
 Call ID: call_rDk5sZ4xipvOyuV8bFZIcxZ1
  Args:
    query: hello, how are you ?
================================= Tool Message =================================
Name: edenai_text_to_speech

https://d14uq1pz7dzsdq.cloudfront.net/76241c92-b3df-4919-9ed5-019fe42